## Data cleaning lesson/exercise from [DataQuest](https://www.dataquest.io/m/136/data-cleaning-walkthrough)

Read CSV datasets into Pandas DataFrame objects that we'll store in a dictionary

In [1]:
import pandas as pd

# NYC high school datasets we'll use, from NYC Open Data (https://data.cityofnewyork.us/Education)
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

# create a dictionary of data file names, minus .csv extension (keys) to corresponding Pandas DataFrames (values) 
data = {}
for data_file in data_files:
    dataset_name = data_file.split(".")[0]
    data[dataset_name] = pd.read_csv(data_file)

Read survey datasets which are not CSV but instead are tab delimited text files with a "windows-1252" encoding:

In [3]:
# read the survey datasets into Pandas DataFrame objects
all_survey = pd.read_csv("survey_all.txt", 
                         delimiter="\t", 
                         encoding="windows-1252")
d75_survey = pd.read_csv("survey_d75.txt", 
                         delimiter="\t", 
                         encoding="windows-1252")

# combine the two surveys into a single DataFrame object
survey = pd.concat((all_survey, d75_survey), axis=0, sort=True)

C:\home\miniconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  import sys


Clean the survey DataFrame by 

1. renaming the "dbn" field to "DBN", and 
2. filtering down to only the fields we'll need for our analysis

In [4]:
# copy the "dbn" column as a column named "DBN"
survey["DBN"] = survey["dbn"]

# create a list of the relevant fields we'll want to filter into our "survey" DataFrame
survey_fields = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]

# filter the DataFrame, assign it into the data dictionary
data["survey"] = survey[survey_fields]

Create a function to help with creating a DBN column for the class_size DataFrame. The function should behave as follows:

- Takes in a number.
- Converts the number to a string using the str() function.
- Pads the string with a zero if the length of the string is only 1 digit.
- Returns the string

In [7]:
def num_to_str(number):
    return str(number).zfill(2)

Create a DBN column in the class_size DataFrame from the "CSD" (padded with a leading zero if necessary) and "SCHOOL CODE" columns.

In [8]:
data["class_size"]["DBN"] = data["class_size"]["CSD"].apply(lambda x: num_to_str(x)) + data["class_size"]["SCHOOL CODE"]

Create a "DBN" column in the "hs_directory" DatFrame (copy the "dbn" column and name it "DBN").

In [10]:
data["hs_directory"].rename(columns={'dbn': 'DBN'}, inplace=True)

Create a total SAT score column from the three component SAT score columns in the `sat_results` DataFrame.

In [11]:
cols = ["SAT Math Avg. Score", 
        "SAT Critical Reading Avg. Score", 
        "SAT Writing Avg. Score"]
for col in cols:
    data["sat_results"][col] = \
        pd.to_numeric(data["sat_results"][col], errors="coerce")

data["sat_results"]["sat_score"] = \
    data["sat_results"]["SAT Math Avg. Score"] +\
    data["sat_results"]["SAT Critical Reading Avg. Score"] +\
    data["sat_results"]["SAT Writing Avg. Score"]

Use a regular expression to pull the lat/lon values from the `Location 1` column of the `hs_directory` DataFrame:

In [23]:
import re

def extract_lat(location):
    
    # get the first part of the lat/lon pair
    lat = re.findall("\(.+\)", location)[0].split()[0]
    
    # clean off the leading parenthesis and the comma
    lat = lat.replace("(", "").replace(",", "")
    
    # return the lat value as a string
    return lat
    
    
def extract_lon(location):
    
    # get the second part of the lat/lon pair
    lon = re.findall("\(.+\)", location)[0].split()[1]
    
    # clean off the trailing parenthesis and the comma
    lon = lon.replace(")", "").replace(",", "")
    
    # return the lon value as a string
    return lon
    
# create lat and lon columns from the lat/lon values extracted from the "Location 1" column
data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(extract_lat) 
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(extract_lon) 

# convert the string values to numeric
data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"],
                                            errors="coerce")
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"],
                                            errors="coerce")

# look to make sure we have what we expect
data["hs_directory"][["Location 1", "lat", "lon"]]

,Location 1,lat,lon
0,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",40.670299,-73.961648
1,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",40.827603,-73.904475
2,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",40.842414,-73.916162
3,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",40.710679,-74.000807
4,"160 20 Goethals Avenue\nJamaica, NY 11432\n(40...",40.718810,-73.806500
5,"3000 East Tremont Avenue\nBronx, NY 10461\n(40...",40.840514,-73.838121
6,"850 Grand Street\nBrooklyn, NY 11211\n(40.7119...",40.711963,-73.940434
7,"345 East 15Th Street\nNew York, NY 10003\n(40....",40.732485,-73.983053
8,"167 01 Gothic Drive\nJamaica, NY 11432\n(40.71...",40.713577,-73.796518
9,207 01 116th Avenue Cambria Heights\nNY 11411\...,40.697807,-73.745858
